# Objetivo
O objetivo deste notebook é demostrar com código as etapas para acessar e utilizar o Llama-2b para a sumarização de um RSE (Relatório de Saúde Eletrônico).

## Autenticação

### Packages

In [ ]:
%%capture
!pip install transformers langchain accelerate # langchain
!pip install gradio

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer
from  langchain import LLMChain, HuggingFacePipeline, PromptTemplate

### HuggingFace - Login

In [ ]:
# Permissão para acessar o modelo
!huggingface-cli login

## Template

In [ ]:
%%capture
# Chamar o modelo Llma-2b
model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)

In [ ]:
%%capture
# Configurar parâmetros
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=3000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

In [ ]:
%%capture
# Usar HuggingFacePipeline para facilitar interação com modelo
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0}) # Facilita passar os parâmetros para o Llama-2b

# Denifir pedido
template = """
              Sumarieze o texto
              ```{text}```
              SUMMARY:
           """
# Passar template ao modelo
prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

## Testa - Sem Interface

In [ ]:
# Conteúdo para ser sumarizado
text = """
1. Histórico Médico:
- Condições médicas pré-existentes: Hipertensão arterial
- Cirurgias anteriores: Apendicectomia em 2005
- Alergias: Nenhuma conhecida
- Medicamentos em uso atualmente: Losartana 50mg uma vez ao dia
2. Registros de Consultas:
- Data e hora da consulta: 02/05/2024, 10:00
- Nome do médico: Dra. Ana Santos
- Diagnóstico: Hipertensão controlada, sem complicações
- Tratamento prescrito: Continuar com a medicação atual, monitorar pressão arterial regularmente
- Recomendações médicas: Dieta balanceada, redução do consumo de sal e prática regular de exercícios
3. Exames e Resultados:
- Tipos de exames realizados: Hemograma completo, perfil lipídico
- Resultados dos exames: Hemoglobina: 14 g/dL, Colesterol total: 190 mg/dL, HDL: 50 mg/dL, LDL: 120 mg/dL
- Data dos exames: 28/04/2024
- Nome do laboratório ou instituição onde os exames foram realizados: Laboratório ABC
"""

## Interface com Gradio

In [ ]:
import gradio as gr
import re

def deploy(text):
    answer = llm_chain.invoke(text)

    # Access the 'text' key from the dictionary
    answer_text = answer['text']

    # Remove unnecessary whitespaces and newlines
    answer_text = re.sub(r'\n\s+', '\n', answer_text.strip())

    # Replace Markdown list items with bullet points
    answer_text = re.sub(r'\n\s*-\s*', '\n* ', answer_text)

    # Replace inline code blocks with backticks
    answer_text = re.sub(r'```', '`', answer_text)

    # Add proper indentation for the summary section
    summary_start = answer_text.find('SUMMARY:')
    answer_text = answer_text[:summary_start] + '\n\nSUMMARY:\n' + answer_text[summary_start + len('SUMMARY:'):].strip()

    return answer_text

In [ ]:
deploy(text)

In [ ]:
demo = gr.Interface(fn=deploy, inputs="text", outputs="text")
demo.launch(share=True)